In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

import pandas as pd

In [2]:
data = pd.read_csv('data/insurance_data_preprocessed.csv')
data.head()

,age,sex,bmi,children,smoker,region,charges
0,19,1,27.900,0,1,0,16884.92400
1,18,0,33.770,1,0,1,1725.55230
2,28,0,33.000,3,0,1,4449.46200
3,33,0,22.705,0,0,2,21984.47061
4,32,0,28.880,0,0,2,3866.85520


In [33]:
test_data = data.sample(frac=0.2, random_state=42)
test_data.reset_index(drop=True, inplace=True)
train_data = data.drop(test_data.index)

len(train_data), len(test_data)

(1070, 267)

In [34]:
test_X = test_data.drop('charges', axis=1).values
test_y = test_data['charges'].values

train_X = train_data.drop('charges', axis=1).values
train_y = train_data['charges'].values

In [36]:
test_X = DataLoader(torch.Tensor(test_X), batch_size=32)
test_y = DataLoader(torch.Tensor(test_y), batch_size=32)

train_X = DataLoader(torch.Tensor(train_X), batch_size=32)
train_y = DataLoader(torch.Tensor(train_y), batch_size=32)

In [75]:
class Model(nn.Module):
    def __init__(self, in_features=6, out_features=1, layers=[12, 6]):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(in_features, layers[0])
        self.fc2 = nn.Linear(layers[0], layers[1])
        self.out = nn.Linear(layers[1], out_features)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        
        return x

torch.manual_seed(42)
model = Model()

In [76]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [77]:
EPOCHS = 1000

for epoch in range(EPOCHS):
    model.train()
    for X, y in zip(train_X, train_y):
        y_pred = model(X)
        loss_train = loss_fn(y_pred.squeeze(), y)
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()
    
    model.eval()
    with torch.inference_mode():
        for X, y in zip(test_X, test_y):
            y_pred = model(X)
            loss_test = loss_fn(y_pred.squeeze(), y)
        
    if epoch % 10 == 0:
        print(f'Epoch {epoch} | Train Loss: {loss_train.item():.4f} | Test Loss: {loss_test.item():.4f}')

Epoch 0 | Train Loss: -611281.4375 | Test Loss: -1450845.7500
Epoch 10 | Train Loss: -2424922368.0000 | Test Loss: -5326841344.0000
Epoch 20 | Train Loss: -18137470976.0000 | Test Loss: -39667941376.0000
Epoch 30 | Train Loss: -56783122432.0000 | Test Loss: -124007792640.0000
Epoch 40 | Train Loss: -124919087104.0000 | Test Loss: -272612769792.0000
Epoch 50 | Train Loss: -227140206592.0000 | Test Loss: -495480864768.0000
Epoch 60 | Train Loss: -366799323136.0000 | Test Loss: -799908298752.0000
Epoch 70 | Train Loss: -546475016192.0000 | Test Loss: -1191507787776.0000
Epoch 80 | Train Loss: -768276758528.0000 | Test Loss: -1674871439360.0000
Epoch 90 | Train Loss: -1034040705024.0000 | Test Loss: -2253996687360.0000
Epoch 100 | Train Loss: -1345453686784.0000 | Test Loss: -2932557742080.0000
Epoch 110 | Train Loss: -1704131035136.0000 | Test Loss: -3714072182784.0000
Epoch 120 | Train Loss: -2111666388992.0000 | Test Loss: -4602010796032.0000
Epoch 130 | Train Loss: -2569649258496.0000 

In [78]:
model.eval()
with torch.inference_mode():
    X, y = next(iter(test_X))[1], next(iter(test_y))[1]
    y_pred = model(X)
    print(f'Actual: {y.item():.4f} | Predicted: {y_pred.item():.4f}')
    loss = loss_fn(y_pred.squeeze(), y)
    print(f'Loss: {loss.item():.4f}')

Actual: 5708.8672 | Predicted: 30175733760.0000
Loss: -172239076982784.0000
